In [ ]:
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.io as spio
import pandas as pd

In [ ]:
load = pd.read_csv('building_energy_storage_simulation/data/preprocessed/electricity_load_profile.csv')['Load [kWh]']
price = pd.read_csv('building_energy_storage_simulation/data/preprocessed/electricity_price_profile.csv')['Day Ahead Auction']
generation_forecast = pd.read_csv('building_energy_storage_simulation/data/preprocessed/solar_generation_profile.csv')['Generation [kWh]']
load = np.array(load)
price = np.array(price)

load = load[200:400]
price = price[200:400]

In [ ]:
n = len(load)
dt = 1

# model parameter initilization
time = range(n)
soc_time = range(n+1)
max_power_charge = 20
max_power_discharge =-20
max_soc = 100
min_soc = 0
soc_init = 50
energy_capacity = 100

m =  AbstractModel()
m.pe_c = Var(time, domain=NonNegativeReals,bounds=(0,value(max_power_charge)))
m.pe_d = Var(time, domain=NegativeReals,bounds=(value(max_power_discharge),0))
m.soc = Var(soc_time, bounds=(value(min_soc),value(max_soc)))

def obj_expression(m):
    return sum([price[i]*(m.pe_c[i] + m.pe_d[i]) for i in time]) 
m.OBJ = Objective(rule=obj_expression, sense=minimize)

def soc_start_rule(m):
    return m.soc[0] == soc_init
m.soc_start = Constraint(rule=soc_start_rule)

def soc_constraint_rule(m,i):
    return m.soc[i+1] ==  float(100) * dt * (m.pe_d[i] + m.pe_c[i]) / energy_capacity + m.soc[i]
m.soc_constraints = Constraint(time, rule=soc_constraint_rule) 

instance = m.create_instance()

solver = SolverFactory('ipopt')
solver.solve(instance,tee=True) 

m = instance
t = [value(time[i])*dt for i in time]
pe = [(value(m.pe_c[i]) + value(m.pe_d[i])) for i in time]

In [ ]:
fig1 = plt.figure()
ax = plt.subplot()
Line1 = ax.plot(time, [(load[i]) for i in time], label='Residual Load')
Line2 = ax.plot(time, [(load[i]+pe[i]) for i in time], label='Load + Battery')
Line3 = ax.plot(time, price, '--', label='price')
plt.ylabel('Load (kW) & Price (Cent per kWh)')
plt.xlabel('Time Step')
ax.legend()
ax.grid()

In [ ]:
plt.subplot()
Line4, = plt.step(t, [(pe[i])for i in time])
plt.ylabel('Power (kW)')
plt.xlabel('Time Step')

In [ ]:
plt.subplot()
Line6, = plt.plot([value(soc_time[i])*dt for i in soc_time], [value(m.soc[i]) for i in soc_time])
Line7, = plt.plot([value(soc_time[i])*dt for i in soc_time], [max_soc for i in soc_time],'--',color='gray')
Line8, = plt.plot([value(soc_time[i])*dt for i in soc_time], [min_soc for i in soc_time],'--',color='gray')
plt.text(1,max_soc-10,'max',color='gray')
plt.text(1,min_soc-10,'min',color='gray')
plt.ylabel('State-of-Charge (%)')
plt.xlabel('Time Step')